In [10]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
base_dir = "/content/drive/MyDrive/Colab Notebooks/datasets/chest_xray"

train_dir = base_dir + "/train"
val_dir   = base_dir + "/val"
test_dir  = base_dir + "/test"

print("Train:", train_dir)
print("Val:", val_dir)
print("Test:", test_dir)


Train: /content/drive/MyDrive/Colab Notebooks/datasets/chest_xray/train
Val: /content/drive/MyDrive/Colab Notebooks/datasets/chest_xray/val
Test: /content/drive/MyDrive/Colab Notebooks/datasets/chest_xray/test


In [12]:
import os

print("Train exists:", os.path.exists(train_dir))
print("Val exists:", os.path.exists(val_dir))
print("Test exists:", os.path.exists(test_dir))


Train exists: True
Val exists: True
Test exists: True


In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Validation & Test: Only rescale
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

val_gen = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

test_gen = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False   # important for evaluation
)


Found 5225 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),

    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')   # Binary classification
])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [15]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 1059s 6s/step - accuracy: 0.8147 - loss: 2.8594 - val_accuracy: 0.5000 - val_loss: 47.2673
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 116s 710ms/step - accuracy: 0.9007 - loss: 0.2685 - val_accuracy: 0.5000 - val_loss: 27.4821
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 116s 709ms/step - accuracy: 0.9043 - loss: 0.2637 - val_accuracy: 0.5000 - val_loss: 23.3245
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 115s 698ms/step - accuracy: 0.9149 - loss: 0.2340 - val_accuracy: 0.6250 - val_loss: 0.5403
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 116s 705ms/step - accuracy: 0.9152 - loss: 0.2294 - val_accuracy: 0.5625 - val_loss: 7.9575
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 116s 705ms/step - accuracy: 0.9254 - loss: 0.1968 - val_accuracy: 0.5000 - val_loss: 0.9487
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 116s 705ms/step - accuracy: 0.9355 - loss: 0.1943 - val_accuracy: 0.5000 - val_loss: 16.2387
Epoch 8/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 116s 710ms/step - accuracy: 0.9174

In [ ]:
# Step 4: Evaluate on test set
test_loss, test_acc = model.evaluate(test_gen)
print("🟢 Test Accuracy:", test_acc)
print("🟢 Test Loss:", test_loss)


20/20 ━━━━━━━━━━━━━━━━━━━━ 172s 9s/step - accuracy: 0.8614 - loss: 0.4183
🟢 Test Accuracy: 0.807692289352417
🟢 Test Loss: 0.42076951265335083


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict on test generator
pred_probs = model.predict(test_gen)
y_pred = (pred_probs > 0.5).astype("int32").flatten()

# True labels
y_true = test_gen.classes

# Classification Report
report_text = classification_report(y_true, y_pred, target_names=['NORMAL', 'PNEUMONIA'])
print(report_text)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(cm)


20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 467ms/step
              precision    recall  f1-score   support

      NORMAL       0.69      0.87      0.77       234
   PNEUMONIA       0.91      0.77      0.83       390

    accuracy                           0.81       624
   macro avg       0.80      0.82      0.80       624
weighted avg       0.83      0.81      0.81       624


Confusion Matrix:
[[204  30]
 [ 90 300]]


In [ ]:
model.save("/content/drive/MyDrive/pneumonia_model.h5")
print("Model saved!")


Model saved!


In [ ]:
import joblib
joblib.dump(history.history, "/content/drive/MyDrive/training_history.pkl")
print("Training history saved!")


Training history saved!


In [ ]:
joblib.dump({
    "test_accuracy": float(test_acc),
    "test_loss": float(test_loss),
    "classification_report": report_text,
    "confusion_matrix": cm.tolist()
}, "/content/drive/MyDrive/evaluation_metrics.pkl")

print("Evaluation metrics saved!")


Evaluation metrics saved!


In [ ]:
import tensorflow as tf
import os

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("/content/drive/MyDrive/pneumonia_model.tflite", "wb") as f:
    f.write(tflite_model)

tflite_size = os.path.getsize("/content/drive/MyDrive/pneumonia_model.tflite") / (1024*1024)
print(f"TFLite Model Size: {tflite_size:.2f} MB")


Saved artifact at '/tmp/tmp_m7130qv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_28')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140509055504976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055509776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055513808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14050905551

In [ ]:
import time

interpreter = tf.lite.Interpreter(model_path="/content/drive/MyDrive/pneumonia_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Pick one image
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

sample = test_gen.filepaths[0]

img = load_img(sample, target_size=(224,224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0) / 255.0

start = time.time()
for _ in range(100):
    interpreter.set_tensor(input_details[0]['index'], x)
    interpreter.invoke()
end = time.time()

avg_ms = ((end - start) / 100) * 1000
print(f"Average Inference Time: {avg_ms:.2f} ms/image")


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Average Inference Time: 29.95 ms/image


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_fp16 = converter.convert()

with open("/content/drive/MyDrive/pneumonia_model_fp16.tflite", "wb") as f:
    f.write(tflite_fp16)

import os
fp16_size = os.path.getsize("/content/drive/MyDrive/pneumonia_model_fp16.tflite") / (1024*1024)
print(f"Float16 Model Size: {fp16_size:.2f} MB")


Saved artifact at '/tmp/tmpyr3k0r_i'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_28')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140509055504976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055509776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055513808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14050905551

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_int8 = converter.convert()

with open("/content/drive/MyDrive/pneumonia_model_int8.tflite", "wb") as f:
    f.write(tflite_int8)

int8_size = os.path.getsize("/content/drive/MyDrive/pneumonia_model_int8.tflite") / (1024*1024)
print(f"INT8 Quantized Model Size: {int8_size:.2f} MB")


Saved artifact at '/tmp/tmp5_q0di4l'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_28')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140509055504976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055512080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055511120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055509776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140509055513808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14050905551

In [ ]:
dummy_input = np.zeros((1, 224, 224, 3), dtype=np.float32)
_ = model.predict(dummy_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [ ]:
last_conv_layer_name = "conv2d_8"


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]

    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    return heatmap.numpy()


In [ ]:
# Pick one test image automatically
img_path = test_gen.filepaths[0]
print("Using image:", img_path)

img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0



Using image: /content/drive/MyDrive/Colab Notebooks/datasets/chest_xray/test/NORMAL/IM-0001-0001.jpeg


In [ ]:
# 🔥 IMPORTANT — Activate the model
dummy_input = np.zeros((1, 224, 224, 3), dtype=np.float32)
_ = model.predict(dummy_input)
print("Model activated!")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Model activated!


In [ ]:
_ = model.predict(img_array)
print("Model called with actual image!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Model called with actual image!


In [3]:
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/pneumonia_model.h5")
print("Model loaded successfully!")



Model loaded successfully!


In [5]:
dummy_input = np.zeros((1, 224, 224, 3), dtype=np.float32)
_ = model.predict(dummy_input)
print("Model activated!")


NameError: name 'np' is not defined

In [ ]:
last_conv_layer_name = "conv2d_8"


In [ ]:
# Force build graph for Grad-CAM
model.build((None, 224, 224, 3))
print("Model graph built!")


Model graph built!


In [ ]:
# new approach:

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image


In [ ]:
last_conv_layer = model.get_layer("conv2d_8")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = tf.keras.models.load_model(
    "/content/drive/MyDrive/pneumonia_model.h5",
    compile=False,
)


In [4]:
model(tf.zeros((1, 224, 224, 3)))
print("MODEL BUILT!")


NameError: name 'tf' is not defined

In [ ]:
last_conv_layer = model.get_layer("conv2d_8")


In [ ]:
grad_model = tf.keras.models.Model(
    inputs=model.input,
    outputs=[last_conv_layer.output, model.output]
)


NameError: name 'model' is not defined

In [ ]:

from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/pneumonia_model.h5")


In [8]:
import os, tensorflow as tf, numpy as np
from tensorflow.keras.models import load_model

# path to your h5 file (change if needed)
h5_path = "/content/drive/MyDrive/pneumonia_model.h5"  # OR use uploaded filename in /content/

# Load without compiling (safer)
model = load_model(h5_path, compile=False)
print("Loaded model object:", model)

# Force a build/pass so Keras creates input/output tensors (fixes Keras3 lazy graph)
_ = model(tf.zeros((1,224,224,3), dtype=tf.float32))
print("Model forced-built with dummy input. model.input exists?:", hasattr(model, 'input'))
print("model.output exists?:", hasattr(model, 'output'))


Loaded model object: <Sequential name=sequential_2, built=True>
Model forced-built with dummy input. model.input exists?: False
model.output exists?: False


In [9]:
from tensorflow.keras import Input, Model

# Create a functional wrapper that feeds through the loaded model
inp = Input(shape=(224,224,3))
out = model(inp)   # run the Keras object as a layer on the Input
func_model = Model(inputs=inp, outputs=out)
print("Functional wrapper created. Inputs/outputs:", func_model.input, func_model.output)

# use func_model for Grad-CAM below
model_for_gradcam = func_model


Functional wrapper created. Inputs/outputs: <KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor_182> <KerasTensor shape=(None, 1), dtype=float32, sparse=False, ragged=False, name=keras_tensor_183>


In [11]:
# -----------------------------
# 100% FIX FOR last_conv ISSUE
# -----------------------------

import tensorflow as tf

use_model = model_for_gradcam if 'model_for_gradcam' in globals() else model

# First try explicit layer name
last_conv_name = "conv2d_8"
last_conv = None

# Try direct fetch
try:
    last_conv = use_model.get_layer(last_conv_name)
    print("Found last conv layer explicitly:", last_conv_name)
except:
    print("Explicit layer not found. Searching automatically...")

# If still None → search automatically
if last_conv is None:
    for layer in reversed(use_model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv = layer
            last_conv_name = layer.name
            print("Automatically selected last conv layer:", last_conv_name)
            break

# Safety check
if last_conv is None:
    raise ValueError("❌ ERROR: No Conv2D layer found in model!")

print("✅ Using last conv layer object:", last_conv)
print("✅ Using last conv layer name:", last_conv_name)

# Now build grad model safely
grad_model = tf.keras.models.Model(
    inputs=use_model.inputs,
    outputs=[last_conv.output, use_model.output]
)

print("Grad model created successfully!")


Explicit layer not found. Searching automatically...


ValueError: ❌ ERROR: No Conv2D layer found in model!

newwwwww **inwww** *italicized text* bold text

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# list the folder where you believe the model is
!ls -lh "/content/drive/MyDrive/Pneumonia_Project" || true
!ls -lh "/content/drive/MyDrive" | grep -i pneumonia || true


Mounted at /content/drive
ls: cannot access '/content/drive/MyDrive/Pneumonia_Project': No such file or directory
-rw------- 1 root root  22M Nov 13 15:00 pneumonia_model_fp16.tflite
-rw------- 1 root root 128M Nov 13 14:49 pneumonia_model.h5
-rw------- 1 root root  11M Nov 13 15:00 pneumonia_model_int8.tflite
-rw------- 1 root root  43M Nov 13 14:51 pneumonia_model.tflite


In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model
h5_path = "/content/drive/MyDrive/pneumonia_model.h5"   # change if filename is different

print("exists:", tf.io.gfile.exists(h5_path))
model = load_model(h5_path, compile=False)
print("Loaded model object type:", type(model))
# Force it to build (Keras3 safe)
_ = model(tf.zeros((1,224,224,3), dtype=tf.float32))
print("Forced build OK")
print("Model summary:")
model.summary()


exists: True
Loaded model object type: <class 'keras.src.models.sequential.Sequential'>
Forced build OK
Model summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [5]:
import tensorflow as tf

use_model = model   # your loaded model

last_conv_name = "conv2d_8"
last_conv = use_model.get_layer(last_conv_name)

print("Using last conv layer:", last_conv.name)

# Build Grad-CAM model
grad_model = tf.keras.models.Model(
    inputs=use_model.inputs,
    outputs=[last_conv.output, use_model.output]
)

print("Grad model successfully created!")


Using last conv layer: conv2d_8


AttributeError: The layer sequential_2 has never been called and thus has no defined output.

In [6]:
import tensorflow as tf
from tensorflow.keras import Input, Model

# Force a call so the model builds
dummy = tf.zeros((1,224,224,3), dtype=tf.float32)
_ = model(dummy)

# Wrap into Functional API — this fixes missing inputs/outputs
inp = Input(shape=(224,224,3))
out = model(inp)
fixed_model = Model(inputs=inp, outputs=out)

print("Functional model created!")
print(fixed_model.summary())


Functional model created!


Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 1)              │    11,169,985 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

None


In [8]:
for layer in fixed_model.layers:
    print(layer.name, layer.__class__.__name__)


input_layer InputLayer
sequential_2 Sequential


In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import Input, Model
import os

h5_path = "/content/drive/MyDrive/pneumonia_model.h5"   # set your path

print("exists:", tf.io.gfile.exists(h5_path))
model = load_model(h5_path, compile=False)
print("Loaded model type:", type(model))

# print original summary
print("\nOriginal model summary:")
model.summary()

# force a build (call once with zeros)
_ = model(tf.zeros((1,224,224,3), dtype=tf.float32))
print("\nForced a dummy call to build the model graph in this session.")

# wrap into a Functional model so inputs/outputs are defined
inp = Input(shape=(224,224,3))
out = model(inp)
fixed_model = Model(inputs=inp, outputs=out)
print("\nFunctional (wrapped) model summary:")
fixed_model.summary()

# save the fixed model (optional but recommended)
fixed_path = "/content/drive/MyDrive/pneumonia_model_fixed.h5"
fixed_model.save(fixed_path, include_optimizer=False)
print("Saved fixed model to:", fixed_path)


exists: True
Loaded model type: <class 'keras.src.models.sequential.Sequential'>

Original model summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 1)              │    11,169,985 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

Saved fixed model to: /content/drive/MyDrive/pneumonia_model_fixed.h5


In [2]:
from google.colab import files
uploaded = files.upload()

Saving pneumonia_model.h5 to pneumonia_model.h5


In [15]:
import os
print(os.listdir())


['.config', 'pneumonia_model.h5', 'drive', 'sample_data']


In [16]:
h5_path = "pneumonia_model.h5"
model = load_model(h5_path, compile=False)


In [17]:
from tensorflow.keras.models import load_model

h5_path = "pneumonia_model.h5"
model = load_model(h5_path, compile=False)
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [20]:
import tensorflow as tf

# Force a forward pass to build the layers
dummy = tf.zeros((1, 224, 224, 3))
_ = model(dummy)

print("Model graph forced successfully.")



Model graph forced successfully.


In [21]:
from tensorflow.keras import Input, Model

inp = Input(shape=(224,224,3))
out = model(inp)
fixed_model = Model(inputs=inp, outputs=out)

fixed_model.summary()


Model: "functional_80"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 1)              │    11,169,985 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

In [22]:
# 1. Build grad_model and create 1 Grad-CAM overlay
import tensorflow as tf, numpy as np, cv2, os
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

# use your wrapped functional model
use_model = fixed_model   # you said you wrapped -> fixed_model

# find last Conv2D layer automatically
last_conv = None
for layer in reversed(use_model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv = layer
        break
if last_conv is None:
    raise RuntimeError("No Conv2D layer found in the model. Show fixed_model.summary()")

print("Last conv layer:", last_conv.name)

# build grad_model
grad_model = tf.keras.models.Model(inputs=use_model.inputs, outputs=[last_conv.output, use_model.output])
print("grad_model ready")

# set sample image path (update if different)
sample_img_path = "/content/sample_test.jpg"   # if you uploaded via files.upload()
if not os.path.exists(sample_img_path):
    # try first file from a generator if you have one
    try:
        sample_img_path = test_gen.filepaths[0]
        print("Using image from test_gen:", sample_img_path)
    except:
        raise FileNotFoundError("No sample image found at /content/sample_test.jpg and test_gen unavailable. Upload one or set path.")

# load & preprocess image
img = image.load_img(sample_img_path, target_size=(224,224))
orig = image.img_to_array(img).astype(np.uint8)        # for overlay (uint8)
inp = np.expand_dims(orig, axis=0).astype(np.float32) / 255.0

# Grad-CAM routine
def generate_gradcam(grad_model, img_input):
    with tf.GradientTape() as tape:
        conv_outputs, preds = grad_model(img_input)
        # for binary sigmoid output; if softmax change index accordingly
        score = preds[:, 0]
    grads = tape.gradient(score, conv_outputs)
    weights = tf.reduce_mean(grads, axis=(0,1,2)).numpy()
    conv_outputs = conv_outputs.numpy()[0]
    cam = np.zeros(conv_outputs.shape[0:2], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * conv_outputs[:,:,i]
    cam = np.maximum(cam, 0)
    if cam.max() > 0:
        cam = cam / cam.max()
    cam = cv2.resize(cam, (224,224))
    return cam

heatmap = generate_gradcam(grad_model, inp)

# overlay
heatmap_uint8 = np.uint8(255 * heatmap)
heatmap_color = cv2.applyColorMap(heatmap_uint8, cv2.COLORMAP_JET)
overlay = cv2.addWeighted(orig, 0.6, heatmap_color, 0.4, 0)

# show and save
plt.figure(figsize=(6,6)); plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)); plt.axis('off'); plt.title("Grad-CAM Overlay"); plt.show()
out_path = "/content/gradcam_overlay.png"
plt.imsave(out_path, cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
print("Saved Grad-CAM overlay to:", out_path)


RuntimeError: No Conv2D layer found in the model. Show fixed_model.summary()

In [3]:
import tensorflow as tf

model = tf.keras.models.load_model("pneumonia_model.h5")


In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # reduces size
tflite_model = converter.convert()

with open("pneumonia_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Saved TFLite model!")


Saved artifact at '/tmp/tmpz_10bnb7'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135672882004432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882005776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882007312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882008080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882005392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882006736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882009424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882010192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882011536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135672882011920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1356728820113

In [5]:

import os
print(os.path.getsize("pneumonia_model.tflite") / 1024 / 1024, "MB")


10.666297912597656 MB


In [6]:
from google.colab import files
files.download("pneumonia_model.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!pip install tensorflowjs


In [2]:
!tensorflowjs_converter \
    --input_format=keras \
    pneumonia_model.h5 \
    tfjs_model/


2025-11-22 06:18:51.994102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763792332.044501    9783 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763792332.070619    9783 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763792332.111511    9783 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763792332.111555    9783 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763792332.111563    9783 computation_placer.cc:177] computation placer alr

In [3]:
model = build_model()
model.load_weights("pneumonia_model.h5")


NameError: name 'build_model' is not defined

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

def build_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.load_weights("pneumonia_model.h5")
print("Weights loaded successfully!")


Weights loaded successfully!


In [7]:
model.save("pneumonia_full_model.h5")
print("Full model saved!")


Full model saved!


In [8]:
from google.colab import files
files.download("pneumonia_full_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
!pip install tensorflowjs

!tensorflowjs_converter \
    --input_format=keras \
    pneumonia_full_model.h5 \
    tfjs_model/


2025-11-22 06:28:42.892512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763792922.911996   12453 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763792922.917966   12453 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763792922.933228   12453 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763792922.933254   12453 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763792922.933258   12453 computation_placer.cc:177] computation placer alr

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,985 (42.61 MB)

 Trainable params: 11,169,537 (42.61 MB)

 Non-trainable params: 448 (1.75 KB)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 470s 3s/step - accuracy: 0.7861 - loss: 2.0302 - val_accuracy: 0.5000 - val_loss: 40.3111
Epoch 2/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 118s 721ms/step - accuracy: 0.9001 - loss: 0.3039 - val_accuracy: 0.5000 - val_loss: 67.6820
Epoch 3/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 120s 728ms/step - accuracy: 0.9041 - loss: 0.2619 - val_accuracy: 0.5000 - val_loss: 4.7501
Epoch 4/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 119s 723ms/step - accuracy: 0.9047 - loss: 0.2859 - val_accuracy: 0.5000 - val_loss: 20.5561
Epoch 5/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 119s 723ms/step - accuracy: 0.9329 - loss: 0.2132 - val_accuracy: 0.6250 - val_loss: 0.7061
Epoch 6/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 117s 711ms/step - accuracy: 0.9055 - loss: 0.2216 - val_accuracy: 0.7500 - val_loss: 0.5404
Epoch 7/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 141s 708ms/step - accuracy: 0.9271 - loss: 0.2033 - val_accuracy: 0.5000 - val_loss: 30.4317
Epoch 8/10
164/164 ━━━━━━━━━━━━━━━━━━━━ 118s 716ms/step - accuracy: 0.9383 

In [1]:
# Step 4: Evaluate on test set
test_loss, test_acc = model.evaluate(test_gen)
print("🟢 Test Accuracy:", test_acc)
print("🟢 Test Loss:", test_loss)

NameError: name 'model' is not defined